In [9]:
from typing import Tuple, List

import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import warnings

sns.set_theme()
from tensorflow import keras

from constants import TRAINING_PARTITIONS, ALL_PARTITIONS, annotation_mapping, amino_acid_mapping, reverse_annotation_mapping
from constants import TYPES, KINGDOMS, METRIC_KINGDOMS, METRIC_TYPES
from metrics.metrics import *
from utils.Dataset import Dataset
from utils.helpers import getDatasetPath
from utils.encoding import categoricalToSequence, oneHotToCategorical, sequenceToCategorical, categoricalToOneHot
from serialization import Serializer

# Load models

In [4]:
run_timestamp = "20211113-0233"
base_path = f"../results/{run_timestamp}/"
cv_models = {}
for k in TRAINING_PARTITIONS:
    cv_models[k] = keras.models.load_model(base_path + f"models/holdout-fold_{k}_final.h5")

# Bootstrap

## Functions

In [34]:
def convertToPerProteinPrediction(prediction: str) -> str:
    if sum([prediction.count(label) for label in "STL"]) > 5:
        type_number = np.argmax([prediction.count('S'), prediction.count('T'), prediction.count('L')])
    else:
        type_number = 3

    return {0: "SP", 1: "TAT", 2: "LIPO", 3: "NO_SP"}[type_number]


In [36]:
def getRelevantData(df: pd.DataFrame, query: str) -> Tuple[np.ndarray, np.ndarray]:
    relevant_data = df.query(query)
    return (list(relevant_data["prediction"]), list(relevant_data["label"]))

def binarize(labels: List[str], label: str) -> np.ndarray:
    array = np.array(labels)
    binarized = array.copy()
    binarized[array == label] = 1
    binarized[array != label] = 0

    return binarized.astype("int")

def bootstrap(test_data: pd.DataFrame, num_bootstraps: int):
    bootstraps = {i: test_data.sample(frac=1, replace=True, axis=0) for i in range(num_bootstraps)}

    metrics = {"mcc": matthews_corrcoef, "precision": precision_score, "recall": recall_score}
    metrics_dict = {}
    for i in tqdm(range(num_bootstraps), leave=False, desc="Computing overall metrics"):
        metrics_dict[i] = {}
        for metric_name, metric in metrics.items():
            # All sequences
            y_pred, y_true = getRelevantData(bootstraps[i], "index == index")
            if metric_name != "mcc":
                value = metric(y_true, y_pred, average="macro")
            else:
                value = metric(y_true, y_pred)
            metrics_dict[i][metric_name] = {
                "overall": {
                    "overall": value
                }
            }

            # By kingdom
            for kingdom in KINGDOMS:
                y_pred, y_true = getRelevantData(bootstraps[i], f"kingdom == '{kingdom}'")
                if metric_name != "mcc":
                    value = metric(y_true, y_pred, average="macro")
                else:
                    value = metric(y_true, y_pred)
                metrics_dict[i][metric_name]["overall"][kingdom] = value

    # Manually exclude non-existing labels (L & T) from eukarya
    for i in tqdm(range(num_bootstraps), leave=False, desc="Excluding non-existent pathways"):
        y_pred, y_true = getRelevantData(bootstraps[i], "kingdom == 'EUKARYA'")
        metrics_dict[i]["precision"]["overall"]["EUKARYA"] = precision_score(y_true, y_pred, average="macro", labels=["NO_SP", "SP"])
        metrics_dict[i]["recall"]["overall"]["EUKARYA"] = recall_score(y_true, y_pred, average="macro", labels=["NO_SP", "SP"])

    # By label
    for i in tqdm(range(num_bootstraps), leave=False, desc="Computing per-label metrics"):
        for label in ["NO_SP", "SP", "LIPO", "TAT"]:
            y_true, y_pred = getRelevantData(bootstraps[i], f"index == index")
            y_true = binarize(y_true, label)
            y_pred = binarize(y_pred, label)

            metrics_dict[i]["mcc"][label] = {"overall": matthews_corrcoef(y_true, y_pred)}
            metrics_dict[i]["precision"][label] = {"overall": precision_score(y_true, y_pred)}
            metrics_dict[i]["recall"][label] = {"overall": recall_score(y_true, y_pred)}

    # By label x kingdom
    for i in tqdm(range(num_bootstraps), leave=False, desc="Computing per-label & kingdom metrics"):
        for label in ["NO_SP", "SP", "LIPO", "TAT"]:
            for kingdom in KINGDOMS:
                y_true, y_pred = getRelevantData(bootstraps[i], f"kingdom == '{kingdom}'")
                y_true = binarize(y_true, label)
                y_pred = binarize(y_pred, label)

                metrics_dict[i]["mcc"][label][kingdom] = matthews_corrcoef(y_true, y_pred)
                metrics_dict[i]["precision"][label][kingdom] = precision_score(y_true, y_pred)
                metrics_dict[i]["recall"][label][kingdom] = recall_score(y_true, y_pred)

    return metrics_dict

## Execution

In [37]:
NUM_BOOTSTRAPS_PER_FOLD = 256

cv_metrics_dict = {}

warnings.filterwarnings("ignore")
dataset = Dataset(getDatasetPath())
for k, model in tqdm(cv_models.items(), desc="Fold progress"):
    test_data = dataset.getFolds([k])

    test_x = np.array([categoricalToOneHot(sequenceToCategorical(seq, amino_acid_mapping), amino_acid_mapping) for seq in test_data["sequence"]])
    predictions = model.predict(test_x)
    y_true = np.array([sequenceToCategorical(seq, annotation_mapping) for seq in test_data["annotation"]])
    y_pred = np.array([oneHotToCategorical(pred) for pred in predictions])

    test_data["prediction"] = np.array([convertToPerProteinPrediction(categoricalToSequence(pred, reverse_annotation_mapping)) for pred in y_pred])
    test_data["label"] = list(test_data.index.get_level_values("type"))

    cv_metrics_dict[k] = bootstrap(test_data, NUM_BOOTSTRAPS_PER_FOLD)


Computing overall metrics: 100%|██████████| 256/256 [00:20<00:00, 12.83it/s]
                                                                            
Excluding non-existent pathways:  98%|█████████▊| 252/256 [00:04<00:00, 60.53it/s]
                                                                                  
Computing per-label metrics:  99%|█████████▉| 254/256 [00:08<00:00, 29.79it/s]
                                                                              
Computing overall metrics: 100%|██████████| 256/256 [00:19<00:00, 13.52it/s]
                                                                            
Excluding non-existent pathways:  99%|█████████▉| 253/256 [00:04<00:00, 60.74it/s]
                                                                                  
Computing per-label metrics: 100%|█████████▉| 255/256 [00:08<00:00, 30.42it/s]
                                                                              
Computing overall metrics: 100%|██████████| 

## Convert to DataFrame

In [29]:
cv_metrics_dict[1][0]["mcc"]["overall"].keys()

dict_keys(['overall', 'EUKARYA', 'ARCHAEA', 'POSITIVE', 'NEGATIVE'])

In [38]:
labels = ["NO_SP", "SP", "TAT", "LIPO"]

cv_final_metrics = pd.DataFrame([
    (partition, bootstrap, metric, label, kingdom, cv_metrics_dict[partition][bootstrap][metric][label][kingdom])
    for bootstrap in range(NUM_BOOTSTRAPS_PER_FOLD)
    for partition in TRAINING_PARTITIONS
    for metric in ["mcc", "precision", "recall"]
    for label in METRIC_TYPES
    for kingdom in METRIC_KINGDOMS
])

cv_final_metrics.columns = ["holdout_fold", "metric", "bootstrap", "label", "kingdom", "value"]
cv_final_metrics.set_index(["holdout_fold", "metric", "bootstrap", "label", "kingdom"])

value
holdout_fold bootstrap metric label   kingdom           
1            0         mcc    TAT     EUKARYA   0.000000
                                      ARCHAEA   0.904684
                                      POSITIVE  0.635228
                                      NEGATIVE  0.805284
                                      overall   0.749301
...                                                  ...
4            255       recall overall EUKARYA   0.960978
                                      ARCHAEA   0.866667
                                      POSITIVE  0.901300
                                      NEGATIVE  0.918991
                                      overall   0.933179

[76800 rows x 1 columns]

In [ ]:
cv_final_metrics

# Store results

In [39]:
Serializer.save(cv_final_metrics, "cv_metrics_per_protein")